## Build a Movie Recommender System with Collaborative Filtering

The dataset we will be using is **Kaggle MovieLens 20M dataset.**

The dataset describes ratings and free-text tagging activities from MovieLens, a movie recommendation service. It contains 20000263 ratings and 465564 tag applications across 27278 movies. These data were created by 138493 users between January 09, 1995 and March 31, 2015. This dataset was generated on October 17, 2016.

Users were selected at random for inclusion. All selected users had rated at least 20 movies.

In this dataset, we are interested in the **rating.csv** file specifically,.

The **rating.csv** file contains ratings of movies by users. It has following columns:

userId, movieId, rating, timestamp.

We are interested in **userId, movieId and rating.** The range of rating is 0~5. The actual min value of ratings in the dataset is 0.5. The actual max value of ratings in the dataset is 5.0.

In order to run the following codes, we firstly need to download the **rating.csv** file in the link provided below. After downloading the file, we need to put the file in the **same directory** as this notebook. 
There is no need for other files in the dataset for building the recommender system. The **rating.csv** file is enough.

Link for the MovieLens 20M dataset: https://www.kaggle.com/grouplens/movielens-20m-dataset

<img src="https://i.imgur.com/AZ4qOOn.png"  width="1000" align="center"/>

In [1]:
import pandas as pd
import numpy as np
from sortedcontainers import SortedList
from tqdm import tqdm

In [2]:
class Recommender_System():
    
    def __init__(self):
        
        # initialize the field variables
        self.user2movie_train={}
        self.movie2user_train={}
        self.rating_train={}
        self.user2movie_test={}
        self.movie2user_test={}
        self.rating_test={}
        self.ratings_average = None
        self.ratings_deviation = None 
        self.users_neigbors = None
    
    
    def load(self,path):
        
        """
        This function loads the file Rating.csv, turns it to a pandas dataframe.
        
        If other files are passed into this function, minor adjustment may be needed.
        """   
        
        df = pd.read_csv(path,header='infer')
        
        return df        
    
    def reframe(self,path):
              
        """
        This function reframes the dataframe.
        
        If other files are passed into the "load" function, minor adjustment may be needed here when reframing.
        
        """
        print("Reframing the data...")
        df = self.load(path).drop(columns=["timestamp"]) 
        
        # set userId index starting from zero
        df['userId'] = df['userId'] - 1
        
        # Reset movieId index to be continuous      
        set_of_movie_ids = sorted(set(df['movieId'].values))    
        reset_movies = self.reset(set_of_movie_ids)       
        df['movieId'] = df['movieId'].apply(lambda r:reset_movies[r])
        
        print("Done.")       
        return df
                
    def reduce(self,df,N,M):
        
        print("Reducing the data...")
        users = self.counter(df['userId'].values)
        movies = self.counter(df['movieId'].values)
        top_users = [x[0] for x in sorted(users.items(),key=lambda x: x[1],reverse=True)[:N]]
        top_movies = [x[0] for x in sorted(movies.items(),key=lambda x: x[1],reverse=True)[:M]]
        df = df[df['userId'].isin(top_users) & df['movieId'].isin(top_movies)]
        
        # reset user and movie index to make it continuous
        
        set_of_user_ids = sorted(set(df['userId'].values))
        set_of_movie_ids = sorted(set(df['movieId'].values))
        reset_users = self.reset(set_of_user_ids)
        reset_movies = self.reset(set_of_movie_ids)
        df['userId'] = df['userId'].apply(lambda r: reset_users[r])
        df['movieId'] = df['movieId'].apply(lambda r: reset_movies[r])
        print("Done.")
        
        return df
        
            
    def mapping_training_set(self,row):
        
        """
        This function creates user2movie, movie2user, and rating maps of the training dataset by row.
        
        """
        
        i = row['userId']
        j = row['movieId']
        if i not in self.user2movie_train:
            self.user2movie_train[i] = [j]
        else:
            self.user2movie_train[i].append(j)

        if j not in self.movie2user_train:
            self.movie2user_train[j] = [i]
        else:
            self.movie2user_train[j].append(i)
            
        self.rating_train[(i,j)] = row['rating']            

        
    def mapping_test_set(self,row):
        
        """
        This function creates user2movie, movie2user, and rating maps of the test dataset by row.
        
        """
        
        i = row['userId']
        j = row['movieId']
        
        # No need to create user2movie_test and movie2user_test, though we can
        # If we want to create user2movie_test and movie2user_test, just copy the pattern in "mapping_training_set" function
        
        if i not in self.user2movie_test:
            self.user2movie_test[i] = [j]
        else:
            self.user2movie_test[i].append(j)

        if j not in self.movie2user_test:
            self.movie2user_test[j] = [i]
        else:
            self.movie2user_test[j].append(i) 
            
 
        self.rating_test[(i,j)] = row['rating']    
        
    def mapping(self,df,train_test_split):
        
        """
        This function uses pd.apply method combined with "mapping_training_set" and "mapping_test_set" function
        
        to create user2movie, movie2user, and rating maps.
        
        """
        
        N = len(set(df['userId'].values)) 
        M = len(set(df['movieId'].values)) 
        again = True
        print("Mapping the data...")
        while again == True:
            # shuffle the dataframe
            df = df.sample(frac=1)
            df_len = len(df)
            df_train = df.iloc[:int(train_test_split*df_len)]
            df_test = df.iloc[int(train_test_split*df_len):]
            # pass it to variables a and b in order to suppress the results
            a = df_train.apply(self.mapping_training_set,axis=1)
            b = df_test.apply(self.mapping_test_set,axis=1)
            L1 = len(list(self.user2movie_train.keys()))
            L2 = len(list(self.user2movie_test.keys()))
            L3 = len(list(self.movie2user_train.keys()))
            L4 = len(list(self.movie2user_test.keys()))
            if L1==N and L2 ==N and L3==M and L4==M:
                # Sort maps, userId and movieId both start from 0
                self.user2movie_train = {x[0]:x[1] for x in tqdm(sorted(self.user2movie_train.items(),key=lambda x:x[0]))}
                self.user2movie_test = {x[0]:x[1] for x in tqdm(sorted(self.user2movie_test.items(),key=lambda x:x[0]))}
                self.movie2user_train = {x[0]:x[1] for x in tqdm(sorted(self.movie2user_train.items(),key=lambda x:x[0]))}
                self.movie2user_train = {x[0]:x[1] for x in tqdm(sorted(self.movie2user_train.items(),key=lambda x:x[0]))}   
                again=False
                print("Done.")
                
    def personalized_weights(self,df,train_test_split,threshold,k_nearest):
        
        """
        This function is the core of collaborative filtering recommender system algorithm.
        
        It defines the personalized weight for each pair of users. The correlation between two users
        
        are defined by Pearson coefficient.
        
        """
        
        self.mapping(df,train_test_split)
        self.ratings_average = [] # Each user's average rating on all movies he or she has rated
        self.ratings_deviation = [] # Each user's deviation on all movies he or she has rated, a list of dictionary
        self.users_neigbors = [] # Each user's k-nearest neigbors' weights. For each user, it is a SortedList object of k-nearest neighbors, in which the information of each neighbor of the user is stored as a tuple (weights,userId)
        N = len(set(df['userId'].values))
        print("Generating personalized weights...")
        for i in tqdm(range(N)):
            movies_for_i = self.user2movie_train[i]
            # set is used for intersection between user i and user j later
            movies_for_i_set = set(movies_for_i)
            # a dictionary where key is the movieId, value is the rating that user i gives to this movie
            ratings_for_i={movie:self.rating_train[(i,movie)] for movie in movies_for_i_set}
            average_for_i = np.mean(list(ratings_for_i.values()))
            # a dictionary of each movie corresponding to a deviation of rating
            deviation_for_i = {movie:(rating-average_for_i) for movie,rating in ratings_for_i.items()}
            deviation_i_values = np.array(list(deviation_for_i.values()))
            # standard deviation for pearson coefficient
            sigma_i = np.sqrt(deviation_i_values.dot(deviation_i_values))
            self.ratings_average.append(average_for_i)
            self.ratings_deviation.append(deviation_for_i)
            # Establish a SortedList object (similar to java)
            sorted_list = SortedList()
            # Compute the weights between two users
            for j in range(N):
                # No need to compute weights for the user itself since we don't use the information of user who needs 
                #to be recommended to recommend the user itself
                if i != j:
                    movies_for_j = self.user2movie_train[j]
                    movies_for_j_set = set(movies_for_j)
                    movie_intersect = (movies_for_i_set & movies_for_j_set)
                    # Abandon if two users have too few common items that they share because in that case not much
                    # useful information will be provided. It could speed up the algorithm.
                    if len(movie_intersect) > threshold:
                        ratings_for_j={movie:self.rating_train[(j,movie)] for movie in movies_for_j_set}
                        average_for_j = np.mean(list(ratings_for_j.values()))
                        # a dictionary of each movie corresponding to a deviation of rating
                        deviation_for_j = {movie:(rating-average_for_j) for movie,rating in ratings_for_j.items()}
                        deviation_j_values = np.array(list(deviation_for_j.values()))
                        sigma_j = np.sqrt(deviation_j_values.dot(deviation_j_values))
                        # Compute the weight between two users with pearson cofficient
                        numerator = np.sum(deviation_for_i[m]*deviation_for_j[m] for m in movie_intersect)
                        denominator = sigma_i*sigma_j
                        w_i_j = numerator/denominator

                        # Note: Since SortedList sorts in ascending order, in order to get highest weights 
                        #(k-nearest) we want, we add a minus sign before the weight. 
                        # Later when doing recommendation, the minus sign should be dropped. We could also try the absolute value.
                        sorted_list.add((-w_i_j,j)) # j is user
                        
                        # Drop the rest of weights. Always keep only k-length neighbors (or less), to save space.
                        if len(sorted_list) > k_nearest:
                            del sorted_list[-1]

            self.users_neigbors.append(sorted_list)
        
        print("Done.")
            
            
    def recommend(self,user_index,movie_index):
        
        """
        This function receives a userId and movieId, then predicts a rating that the user would give to the movie.
        
        In other words, this function proceeds actual recommendation.
        
        """
        
        numerator = 0
        denominator = 0
        for weight, user_j in self.users_neigbors[user_index]:
            # If user_j happens not to rate the movie_j that user_i rates, then go to the Exception, which directly passes
            # We could also "find out " if user_j has rated the movie_j, but that would cost the time of finding items in
            # dictionary, which is not efficient. Therefore, we use try except statement.
            
            try:
                # add a second minus sign to weight to make it positive again
                numerator+=(-weight)*(self.ratings_deviation[user_j][movie_index])
                denominator+= abs(weight)

            except:

                pass
        # There might be an extreme case that this user's all neighbor users have not rated the movie that is supposed to
        # be recommended to this user. In this case, the recommendation of this movie would have to be the average of this  
        # user's rating.
        if denominator == 0:
            prediction = self.ratings_average[user_index]
        
        else:
            prediction = (numerator/denominator)+ self.ratings_average[user_index]

        # ADD boundary (0.5-5) here

        return prediction
    
    def root_mean_square_error(self,predictions,labels):
        
        """
        This function receives the predicted ratings and the true ratings,
        
        and then computes the root mean square error of the prediction.
        
        """
        
        predictions = np.array(predictions)
        labels = np.array(labels)        
        rmse = np.sqrt(np.mean((predictions - labels)**2))
        
        return rmse
        
        
    def reset(self,obj):
        
        """
        helper function: Reset index of dataframe.
        
        """
        
        reset_index = {}
        count = 0
        for i in obj:
            reset_index[i] = count
            count+=1
        
        return reset_index
  
                
    def counter(self,obj):
        
        """
        helper function: Count the object of interest.
        
        """
        
        counter = {}
        
        for item in obj:
            
            if item not in counter:
                
                counter[item] = 0
            
            counter[item]+=1
        
        return counter       
        
        
        
        



#### Substantiate the recommender system class

In [3]:
recommender_system = Recommender_System()

#### Preprocess and reframe the dataframe

In [4]:
df_reframed = recommender_system.reframe("./rating.csv")

Reframing the data...
Done.


#### Reduce the dataframe to top 1000 users and 200 movies for the purpose of illustration

 We choose top 1000 users who have rated the most movies, and top 200 movies which have been rated the most.

In [5]:
df_reduced = recommender_system.reduce(df_reframed,N=1000,M=200)

Reducing the data...
Done.


/Users/liangchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/liangchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Compute personalized weights

In order to do recommendation, the personalized weights are computed. First the dataset is split into training and test set with a ratio of 4:1. Then, a threshold = 5 is set for the least
number of common movies that two users have rated. If the number of movies that two users both have rated is less than 5, then there is no need to compute the weights between these two users share too little information. If two
users share too little information, it's not accurate enough to predict one user's behavior based on another.

A k-nearest neighbor value = 25 is also set for users that are most in common. When doing the prediction on one user, we would like to use the top 25 neighboring users who are the most similar to this user. We could also use all the other users to do prediction, but by setting an adequate k-nearest neighbor value, we can speed up the algorithm as well as reserve the accuracy of prediction. k_nearest=25 is enough to do the recommendation.

In [6]:
recommender_system.personalized_weights(df_reduced,train_test_split=0.8,threshold=5,k_nearest=25)

Mapping the data...


  0%|          | 0/1000 [00:00<?, ?it/s]/Users/liangchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:207: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


Done.
Generating personalized weights...


100%|██████████| 1000/1000 [04:00<00:00,  4.11it/s]

Done.


#### Predict the rating

Finally, we do the recommendation by predicting the rating that a user would give to a movie.
If the predicted rating is high, then we recommend this movie to the user, vice versa.

In [7]:
rating_test = recommender_system.rating_test
predictions = []
labels = []
for (user,movie),rating in rating_test.items():    
    prediction = recommender_system.recommend(int(user),int(movie))
    predictions.append(prediction)
    labels.append(rating)

#### Compute root mean square error

There isn't a fix method for evaluating the collaborative filtering recommender system. In this project, we would like to evaluate the model by computing the root mean square error (RMSE) between the predicted rating and the true rating on the test dataset. 
The RMSE is around 0.77, which is a pretty good result for the collaborative filtering recommender system. The RMSE benchmark for recommender system is around 0.9.

In [8]:
rmse = recommender_system.root_mean_square_error(predictions,labels)
rmse

0.7709174962124318